# 📊 Análisis Topológico de Interacciones Gestuales UCI

Este notebook analiza los patrones de interacción en el sistema de comunicación gestual hospitalaria.

## Métricas Calculadas:
- Degree Centrality
- Betweenness Centrality  
- Closeness Centrality
- Network Density
- Network Diameter
- Average Path Length

In [ ]:
import json
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Configuración de estilo
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

## 1. Cargar Datos de Interacción

In [ ]:
# Datos de ejemplo - reemplazar con exportación desde frontend
sample_data = {
    "nodes": [
        {"id": "T1", "label": "Alerta respiratoria", "color": "#EF4444"},
        {"id": "T2", "label": "Alerta cardíaca", "color": "#F59E0B"},
        {"id": "T3", "label": "Fármacos urgentes", "color": "#8B5CF6"},
        {"id": "T4", "label": "Material estéril", "color": "#06B6D4"},
        {"id": "T5", "label": "Equipo quirúrgico", "color": "#10B981"},
    ],
    "edges": [
        {"from": "T1", "to": "T3", "weight": 5},
        {"from": "T1", "to": "T2", "weight": 3},
        {"from": "T2", "to": "T3", "weight": 2},
        {"from": "T3", "to": "T4", "weight": 4},
        {"from": "T4", "to": "T5", "weight": 1},
    ]
}

print(f"Nodos: {len(sample_data['nodes'])}")
print(f"Edges: {len(sample_data['edges'])}")

## 2. Construir Grafo NetworkX

In [ ]:
# Crear grafo dirigido
G = nx.DiGraph()

# Añadir nodos con atributos
for node in sample_data['nodes']:
    G.add_node(node['id'], label=node['label'], color=node['color'])

# Añadir edges con pesos
for edge in sample_data['edges']:
    G.add_edge(edge['from'], edge['to'], weight=edge['weight'])

print(f"Grafo creado: {G.number_of_nodes()} nodos, {G.number_of_edges()} edges")
print(f"Dirigido: {G.is_directed()}")

## 3. Métricas de Centralidad

In [ ]:
# Calcular métricas de centralidad
degree_cent = nx.degree_centrality(G)
betweenness_cent = nx.betweenness_centrality(G)
closeness_cent = nx.closeness_centrality(G)

# Crear DataFrame
centrality_df = pd.DataFrame({
    'Node': list(degree_cent.keys()),
    'Label': [G.nodes[n]['label'] for n in degree_cent.keys()],
    'Degree': list(degree_cent.values()),
    'Betweenness': list(betweenness_cent.values()),
    'Closeness': list(closeness_cent.values())
})

centrality_df = centrality_df.sort_values('Degree', ascending=False)
print("\n📊 Métricas de Centralidad:")
print(centrality_df)

## 4. Visualización del Grafo

In [ ]:
plt.figure(figsize=(14, 10))

# Layout
pos = nx.spring_layout(G, k=2, iterations=50)

# Colores de nodos
node_colors = [G.nodes[n]['color'] for n in G.nodes()]

# Tamaños proporcionales a degree centrality
node_sizes = [3000 * degree_cent[n] + 500 for n in G.nodes()]

# Dibujar nodos
nx.draw_networkx_nodes(G, pos, node_color=node_colors, node_size=node_sizes, alpha=0.9)

# Dibujar edges con grosor proporcional al peso
edges = G.edges()
weights = [G[u][v]['weight'] for u, v in edges]
nx.draw_networkx_edges(G, pos, width=[w * 0.5 for w in weights], alpha=0.6, 
                       arrows=True, arrowsize=20, arrowstyle='->')

# Labels
labels = {n: G.nodes[n]['label'][:15] for n in G.nodes()}
nx.draw_networkx_labels(G, pos, labels, font_size=9, font_weight='bold')

plt.title('Grafo de Interacciones Gestuales UCI', fontsize=16, fontweight='bold')
plt.axis('off')
plt.tight_layout()
plt.show()

## 5. Métricas Globales del Grafo

In [ ]:
# Calcular métricas globales
density = nx.density(G)

# Convertir a no dirigido para algunas métricas
G_undirected = G.to_undirected()

try:
    diameter = nx.diameter(G_undirected)
    avg_path_length = nx.average_shortest_path_length(G_undirected)
except:
    diameter = "N/A (grafo no conexo)"
    avg_path_length = "N/A"

print("\n🌐 Métricas Globales:")
print(f"  Densidad: {density:.3f}")
print(f"  Diámetro: {diameter}")
print(f"  Longitud de camino promedio: {avg_path_length}")
print(f"  Número de componentes conexas: {nx.number_connected_components(G_undirected)}")

## 6. Distribución de Grados

In [ ]:
# Calcular grados
degrees = dict(G.degree())
in_degrees = dict(G.in_degree())
out_degrees = dict(G.out_degree())

# Visualización
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Total degree
axes[0].bar(range(len(degrees)), list(degrees.values()), color='skyblue')
axes[0].set_xticks(range(len(degrees)))
axes[0].set_xticklabels(list(degrees.keys()), rotation=45)
axes[0].set_title('Total Degree', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Degree')

# In-degree
axes[1].bar(range(len(in_degrees)), list(in_degrees.values()), color='lightcoral')
axes[1].set_xticks(range(len(in_degrees)))
axes[1].set_xticklabels(list(in_degrees.keys()), rotation=45)
axes[1].set_title('In-Degree', fontsize=14, fontweight='bold')
axes[1].set_ylabel('In-Degree')

# Out-degree
axes[2].bar(range(len(out_degrees)), list(out_degrees.values()), color='lightgreen')
axes[2].set_xticks(range(len(out_degrees)))
axes[2].set_xticklabels(list(out_degrees.keys()), rotation=45)
axes[2].set_title('Out-Degree', fontsize=14, fontweight='bold')
axes[2].set_ylabel('Out-Degree')

plt.tight_layout()
plt.show()

## 7. Conclusiones

Este análisis topológico permite identificar:
- **Nodos centrales**: Comandos más utilizados en secuencias
- **Patrones de flujo**: Transiciones más comunes
- **Nodos puente**: Comandos que conectan diferentes grupos

Para análisis más profundos:
- Ver notebook 02: Detección de comunidades
- Ver notebook 03: Modelos de difusión
- Ver notebook 04: Análisis de resiliencia